In [115]:
import sys
#Path to Trane for imports
sys.path.append('/Users/Alexander/Documents/Trane/Trane__HDI_REPO')
import pandas as pd
import trane
import json
import random
import datetime


In [101]:
def file_to_table_meta(filepath):
    return trane.TableMeta(json.loads(open(filepath).read()))

In [102]:
path_to_datasets = '../Trane__Local_Misc/Formatted Datasets/'
taxi_data_df = pd.read_csv(path_to_datasets + 'NYC Taxi/taxi.csv')
taxi_table_meta = file_to_table_meta(path_to_datasets + 'NYC Taxi/taxi_meta.json')

In [103]:
NUM_PROBLEMS_TO_GENERATE = 10

In [104]:
def generate_probs_and_nl(entity_id_column,
                            label_generating_column,
                            time_column,
                            table_meta,
                            filter_column,
                            is_pick_random_problems = True):
    generator = trane.PredictionProblemGenerator(table_meta, entity_id_column, label_generating_column, time_column, filter_column)
    probs = []

    all_probs = list(generator.generate())

    if is_pick_random_problems:
        random.shuffle(all_probs)

    for idx, prob in enumerate(all_probs):
        probs.append(prob)
        if idx + 1 == NUM_PROBLEMS_TO_GENERATE:
            break
    prediction_problems_json = trane.prediction_problems_to_json_file(
        probs, table_meta, entity_id_column, label_generating_column, time_column, "prediction_problems.json")

    nl_descrips = trane.generate_nl_description(
        probs, table_meta, entity_id_column, label_generating_column, time_column, trane.ConstantIntegerCutoffTimes(0))
    return probs, nl_descrips

# Trane
Below is the code for generating prediction problems using Trane.

In [105]:
entity_id_column = 'id'
label_generating_column = 'passenger_count'
time_column = 'pickup_datetime'
filter_column = 'id'
table_meta = taxi_table_meta
probs, nl_descrips = generate_probs_and_nl(entity_id_column, label_generating_column,
                     time_column, table_meta, filter_column)
prediction_problems_json = trane.prediction_problems_to_json_file(
    probs, table_meta, entity_id_column, label_generating_column, time_column, "prediction_problems.json")

In [106]:
def convert(str, format = None):
    return datetime.strptime(str, format)
print(taxi_data_df[time_column].head(5))

0    2016-06-30 23:59:58
1    2016-06-30 23:59:53
2    2016-06-30 23:59:47
3    2016-06-30 23:59:41
4    2016-06-30 23:59:33
Name: pickup_datetime, dtype: object


#### Note
The code below converts the data in the time column to datetime objects. 

In [107]:
taxi_data_df[time_column] = taxi_data_df[time_column].apply(convert, format = "%Y-%m-%d %X")

#### Warning!
The code cell below, takes a few minutes to execute.

In [108]:
entity_to_data_dict = trane.df_group_by_entity_id(taxi_data_df, entity_id_column)

#### Note
The code below samples NUM_ENTITIES entities from the dict. The dict above is large, containing over 6e5 keys.

In [109]:
NUM_ENTITIES = 10 
entity_to_data_dict = {key: entity_to_data_dict[key] for key in list(entity_to_data_dict)[:NUM_ENTITIES]}

In [116]:
datetime_constant_cutoff = datetime.date(2000, 1, 1)

In [117]:
entity_to_data_and_cutoff_dict = trane.ConstantDatetimeCutoffTime(datetime_constant_cutoff).generate_cutoffs(entity_to_data_dict)

# CONTINUE HERE

In [118]:
labeler = trane.Labeler()
output = labeler.execute(entity_to_data_and_cutoff_dict, "prediction_problems.json")

TypeError: '<' not supported between instances of 'str' and 'int'

# Feature Tools
Below is the code for generating features using DFS from featuretools.

In [25]:
import featuretools as ft

In [26]:
entities = {
    "taxi_data": (taxi_data_df, "id")
}
relationships = None

In [27]:
feature_matrix, feature_definitions = ft.dfs(entities = entities,
      relationships = relationships,
      target_entity = "taxi_data")

# ATM
Below is the code for automatically tuning the model.